<a href="https://colab.research.google.com/github/Saldanhaa7/CursoEngDados/blob/master/Aula_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



*   Extrair dados públicos, de compras governamentais
*   Tratar os dados
*   Armazenar os dados em um arquivo






# Extract


In [2]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=7c58099d07ac5155d9081463803203f8a966f6f51f5a3fdd596e76246c93ac46
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [18]:
import requests
import io
import pandas as pd
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [4]:
url = 'http://compras.dados.gov.br/servicos/v1/servicos.csv?descricao=educação'

In [5]:
data = requests.get(url).content
type(data)

bytes

In [19]:
parse_data = io.StringIO(data.decode('utf-8'))
parse_data

In [20]:
df = pd.read_csv(parse_data)
df.head()

,Código,Descrição,Unidade medida,CPC,Seção,Divisão,Grupo,Classe,Subclasse
0,14311,Orientação / Educação - Atividade Física ...,UNHOMEM/H MÊS,9659,"9: SERVIÇOS DA COMUNIDADE, PESSOAIS E SOCIAIS ...","96: SERVIÇOS RECREACIONAIS, CULTURAIS E ESPORT...",965: SERVIÇOS RELACIONADOS COM ESPORTES E SERV...,9659: OUTROS ESPORTES E SERVIÇOS DE ESPORTES R...,NaN
1,15946,Serviço Penitenciário - Reeducação e Reabilita...,UN,979,"9: SERVIÇOS DA COMUNIDADE, PESSOAIS E SOCIAIS ...",97: OUTROS SERVIÇOS ...,979: OUTROS SERVIÇOS DIVERSOS/MISCELÂNEA ...,NaN,NaN
2,18481,Consultoria e Assessoria - Educação ...,UN,8319,"8: SERVIÇOS DE ARQUITETURA, DESENHOS TÉCNICOS,...","83: OUTROS SERVIÇOS DE NEGÓCIOS, TÉCNICOS E PR...",831: SERVIÇOS DE CONSULTORIA E DE GERÊNCIA/GES...,"8319: OUTROS SERVIÇOS DE GERÊNCIA/GESTÃO, EXCE...",NaN
3,19321,Curso / Treinamento Educação - Distância ...,UN,929,"9: SERVIÇOS DA COMUNIDADE, PESSOAIS E SOCIAIS ...",92: SERVIÇOS DA INSTRUÇÃO/ EDUCAÇÃO/ ENSINO ...,929: OUTROS SERVIÇOS DE EDUCAÇÃO E TREINAMENTO...,NaN,NaN


In [8]:
spark = SparkSession.builder.appName("Aula1").getOrCreate()

In [23]:
type(df)

pyspark.sql.dataframe.DataFrame

In [22]:
df = spark.createDataFrame(df)

#Transform

In [12]:
df.printSchema()

root
 |-- Código: long (nullable = true)
 |-- Descrição: string (nullable = true)
 |-- Unidade medida: string (nullable = true)
 |-- CPC: long (nullable = true)
 |-- Seção: string (nullable = true)
 |-- Divisão: string (nullable = true)
 |-- Grupo: string (nullable = true)
 |-- Classe: string (nullable = true)
 |-- Subclasse: double (nullable = true)



In [13]:
df.show(50)

+------+--------------------+--------------+----+--------------------+--------------------+--------------------+--------------------+---------+
|Código|           Descrição|Unidade medida| CPC|               Seção|             Divisão|               Grupo|              Classe|Subclasse|
+------+--------------------+--------------+----+--------------------+--------------------+--------------------+--------------------+---------+
| 14311|Orientação / Educ...| UNHOMEM/H MÊS|9659|9: SERVIÇOS DA CO...|96: SERVIÇOS RECR...|965: SERVIÇOS REL...|9659: OUTROS ESPO...|      NaN|
| 15946|Serviço Penitenci...|            UN| 979|9: SERVIÇOS DA CO...|97: OUTROS SERVIÇ...|979: OUTROS SERVI...|                 NaN|      NaN|
| 18481|Consultoria e Ass...|            UN|8319|8: SERVIÇOS DE AR...|83: OUTROS SERVIÇ...|831: SERVIÇOS DE ...|8319: OUTROS SERV...|      NaN|
| 19321|Curso / Treinamen...|            UN| 929|9: SERVIÇOS DA CO...|92: SERVIÇOS DA I...|929: OUTROS SERVI...|                 NaN|   

In [16]:
df.columns

['Codigo',
 'Descricao',
 'Unidade_medida',
 'CPC',
 'Secao',
 'Divisao',
 'Grupo',
 'Classe',
 'Subclasse']

In [15]:
df = (df.withColumnRenamed("Código", "Codigo")
.withColumnRenamed("Descrição", "Descricao")
.withColumnRenamed("Unidade medida", "Unidade_medida")
.withColumnRenamed("Seção", "Secao")
.withColumnRenamed("Divisão", "Divisao")
      )

In [24]:
df = df.select([x.lower() for x in df.columns])
df.printSchema()

root
 |-- código: long (nullable = true)
 |-- descrição: string (nullable = true)
 |-- unidade medida: string (nullable = true)
 |-- cpc: long (nullable = true)
 |-- seção: string (nullable = true)
 |-- divisão: string (nullable = true)
 |-- grupo: string (nullable = true)
 |-- classe: string (nullable = true)
 |-- subclasse: double (nullable = true)



In [25]:
from google.colab import drive

In [26]:
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
dir = "/content/drive/MyDrive/xpe/data"

In [30]:
df.write.mode("overwrite").csv(f"{dir}/compras.csv")

In [33]:
print(spark.read.csv("/content/drive/MyDrive/xpe/data/compras.csv").head())

Row(_c0='18481', _c1='Consultoria e Assessoria - Educação', _c2='UN', _c3='8319', _c4='8: SERVIÇOS DE ARQUITETURA, DESENHOS TÉCNICOS, PESQUISA E DESENVOLVIMENTO, NEGÓCIO E PRODUÇÃO', _c5='83: OUTROS SERVIÇOS DE NEGÓCIOS, TÉCNICOS E PROFISSIONAIS', _c6='831: SERVIÇOS DE CONSULTORIA E DE GERÊNCIA/GESTÃO', _c7='8319: OUTROS SERVIÇOS DE GERÊNCIA/GESTÃO, EXCETO OS DE PROJETOS DECONSTRUÇÃO', _c8='NaN')
